In [65]:
import email
from email import policy
from email.parser import BytesParser

from glob import glob

import xmlrpc.client
import json
import requests

In [63]:

# Path to your .eml file
file_path = glob('../mails/*.eml')[-2]
file_path


'../mails/Esto es un test.eml'

In [64]:



# Open and read the .eml file
with open(file_path, 'rb') as file:
    # Parse the .eml file using BytesParser
    msg = BytesParser(policy=policy.default).parse(file)

# Extracting basic information from the email
subject = msg['subject']
from_email = msg['from']
to_email = msg['to']
date = msg['date']
body = ""

# Extract the body of the email
if msg.is_multipart():
    # If the email has multiple parts (attachments, text, etc.), iterate over the parts
    for part in msg.iter_parts():

        # If it is a multipart in itself
        if part.is_multipart():
            for sub_part in part.iter_parts():
                if sub_part.get_content_type() == 'text/plain':
                    body = sub_part.get_payload(decode=True).decode(sub_part.get_content_charset())        
        else:
            # Look for the plain text part
            if part.get_content_type() == 'text/plain':
                body = part.get_payload(decode=True).decode(part.get_content_charset())
else:
    # If the email is not multipart, get the payload directly
    body = msg.get_payload(decode=True).decode(msg.get_content_charset())

body = body.split('--')[0]

# Display the extracted information
print("Subject:", subject)
print("From:", from_email)
print("To:", to_email)
print("Date:", date)
print("Body:", body)


Subject: Esto es un test
From: Contacto PacificLabs <contacto@pacificlabs.cl>
To: "Santiago Larrain E." <santiagolarrain@gmail.com>
Date: Fri, 08 Nov 2024 11:59:34 -0300
Body: Esto es una prueba, para ver como funciona.

Pero soy santiago escribiendole a Santiago.





In [51]:
msg.is_multipart()

True

In [52]:
partes = []
for part in msg.iter_parts():
    partes.append(part)

In [53]:
len(partes)

11

In [60]:
for part in partes[0].iter_parts():
    print (part.get_content_type())

text/plain
text/html


In [47]:
for part in partes:
    print (part.get_content_type())
    print (part.is_multipart())

text/plain
False
text/html
False


In [57]:
partes[0].is_multipart()

True

In [59]:
partes[0].iter_parts()

<generator object MIMEPart.iter_parts at 0x7d6b96be9240>

In [68]:


url = 'http://192.168.1.122:8069'
db = 'pacificlabs'
username = 'santiagolarrain@pacificlabs.cl'
password = 'sl2012le2187'
common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(url))
common.version()

models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(url))

uid = common.authenticate(db, username, password, {})

rv = models.execute_kw(db, uid, password, 'crm.lead', 'search', [[['email_from', '=', "santiagolarrain@gmail.com"]]])

In [77]:
type(rv[0])

int

In [83]:
contenido = f"""
== PRUEBA NOTEBOOK == 
<br><br>
Subject:{subject}
From:{from_email}
To:{to_email}
Date:{date}

{body}"""
print(contenido)


== PRUEBA NOTEBOOK == 
<br><br>
Subject:Esto es un test
From:Contacto PacificLabs <contacto@pacificlabs.cl>
To:"Santiago Larrain E." <santiagolarrain@gmail.com>
Date:2024-10-21 21:30:30

Esto es una prueba, para ver como funciona.

Pero soy santiago escribiendole a Santiago.





In [79]:
date = '2024-10-21 21:30:30'

In [84]:
            models.execute_kw(
                    db, 
                    uid, 
                    password, 
                    'mail.message', 
                    'create', 
                    [
                        { 
                         'model': 'crm.lead', 
                         'res_id': rv[0], 
                         'body': contenido, 
                         'author_id': uid, 
                         'create_date': date, 
                         'date': date, 
                         'write_date': date, 
                         'attachment_ids':[]
                         }
                        ]
                    )


2584